In [183]:
import numpy as np
import pandas as pd
import random as random
import matplotlib.pyplot as plt
from datetime import datetime
import wrds
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

db = wrds.Connection(wrds_username = 'anita97') 

Loading library list...
Done


In [184]:
#import stocks
'''s = db.raw_sql("select a.permno, a.date, \
                  a.ret, a.shrout, a.prc \
                  from crsp.msf as a \
                  join crsp.msenames as b \
                  on a.permno = b.permno and b.namedt<=a.date \
                  and a.date<=b.nameendt \
                  where b.shrcd between (10) and (11)and \
                  a.date >='2000-01-01' and a.date<='2019-12-31'\
                  and b.exchcd between 1 and 2")'''

's = db.raw_sql("select a.permno, a.date,                   a.ret, a.shrout, a.prc                   from crsp.msf as a                   join crsp.msenames as b                   on a.permno = b.permno and b.namedt<=a.date                   and a.date<=b.nameendt                   where b.shrcd between (10) and (11)and                   a.date >=\'2000-01-01\' and a.date<=\'2019-12-31\'                  and b.exchcd between 1 and 2")'

In [185]:
#s.to_csv('as6.csv')
stocks = pd.read_csv('as6.csv', index_col=0) # read the file
#stocks=stocks.drop(['shrcd','exchcd'],axis=1)

For each value we want one value for each month. Firstly we trasform data column in month-year(format). Then, for each stock we chack that there is only one value for each month:

In [186]:
stocks.date = pd.DatetimeIndex(stocks.date) # data format
stocks.date = stocks.date.dt.to_period('M') # data year-month
stocks.head() # see data format

,permno,date,ret,shrout,prc
0,10001.0,2009-12,0.162621,4361.0,10.3000
1,10001.0,2010-01,-0.018932,4361.0,10.0600
2,10001.0,2010-02,-0.000656,4361.0,10.0084
3,10001.0,2010-03,0.020643,4361.0,10.1700
4,10001.0,2010-04,0.124385,6070.0,11.3900


PERMNO is a unique stock level identifier. While most of the companies have one class shares, some companies have more than one class shares traded at different prices, and this is the reason why a company can have more than one PERMNOs.

We utilize permno as index:

In [187]:
stocks.sort_values(by='permno')
stocks = stocks.drop_duplicates(['permno','date']) # delete duplicates
stocks = stocks.set_index('permno') # set permno as index
stocks.head()

,date,ret,shrout,prc
permno,,,,
10001.0,2009-12,0.162621,4361.0,10.3000
10001.0,2010-01,-0.018932,4361.0,10.0600
10001.0,2010-02,-0.000656,4361.0,10.0084
10001.0,2010-03,0.020643,4361.0,10.1700
10001.0,2010-04,0.124385,6070.0,11.3900


We have to delete all the stocks which have less than 240 observations. In order to do that, we can group all the stocks on permno (which identify one stock) and then count how many observations we have for each stock:

In [188]:
#stocks = stocks[~stocks.ret.isna()] # delete null returns

In [189]:
count = []
count = stocks.groupby('permno').count()
# now in each column we have the number of observations. We can rename one of them
count = count.rename(columns={"date":"number of observations"})
count = count[['number of observations']]
print(f"We have {count.shape[0]} different stocks. Each has a certain number of observations:")
count.head()


We have 4118 different stocks. Each has a certain number of observations:


,number of observations
permno,
10001.0,92
10028.0,140
10042.0,98
10051.0,233
10066.0,28


Now we delete  stocks with less than 240 observations:

Note: This is equal to have only stocks with preciselt 240 observations. Because we want that they are traded EVERY mounth for 20 years: 

20 years*12 mounths = 240 observation. 

In [190]:
decision = count[['number of observations']]!=240 # delete if true
#decision = decision.dropna() # drop NaN
decision.head()

,number of observations
permno,
10001.0,True
10028.0,True
10042.0,True
10051.0,True
10066.0,True


In [191]:
decision_true = decision[decision['number of observations']] # keep only true
print(f"We delete {decision_true.shape[0]} stocks.")
stocks_del = stocks.drop(index=decision_true.index) # drop
stocks_del.head()

We delete 3479 stocks.


,date,ret,shrout,prc
permno,,,,
10145.0,2000-01,-0.167931,789233.0,48.0000
10145.0,2000-02,0.006510,795134.0,48.1250
10145.0,2000-03,0.094805,796591.0,52.6875
10145.0,2000-04,0.062871,796591.0,56.0000
10145.0,2000-05,-0.020089,798161.0,54.6875


We should have 639 stocks:

In [192]:
# we have deleted decision_true
# we are left with the rest of decision
num_deleted = int(decision_true.count())
num_left = int(decision.count()-decision_true.count())
print(f"We have deleted {num_deleted} and we are left with {num_left} stocks")

We have deleted 3479 and we are left with 639 stocks


In [193]:
# import risk free rate
risk_free = db.raw_sql("select mcaldt as date, tmytm as rf from crsp.tfz_mth_rf where kytreasnox = 2000001 and mcaldt>='2000-01-01' and mcaldt<='2019-12-31'")
risk_free_m = np.exp(np.divide(risk_free.rf, 1200)) -1
rf = pd.DataFrame([risk_free.date, risk_free_m], index=['date', 'rf']).T

# import market
market = db.raw_sql("select date, vwretd from crsp.msi where date>='2000-01-01' and date<='2019-12-31'")
market_m = pd.DataFrame([market.date, market.vwretd.values], index=['date', 'mkt_ret']).T
rm = market_m.set_index('date')

In [194]:
rf.date = pd.DatetimeIndex(rf.date) # data format
rf.date = rf.date.dt.to_period('M') # data year-month
#rf = rf.drop('index')
rf = rf.set_index('date')
rf.head()

,rf
date,
2000-01,0.00449835
2000-02,0.00443834
2000-03,0.00489188
2000-04,0.00440126
2000-05,0.00390314


In [195]:
rm.index = pd.DatetimeIndex(rm.index) # data format
rm.index = rm.index.to_period('M') # data year-month
rm.head()

,mkt_ret
date,
2000-01,-0.0396243
2000-02,0.0317784
2000-03,0.0535007
2000-04,-0.059527
2000-05,-0.0388696


We set a multiple index for the stocks:

In [196]:
# change index: first permno then date
ss_permno = stocks_del.reset_index()
ss_permno = ss_permno.set_index(['permno','date']) # multiple index
ss_permno = ss_permno.sort_values(by=['permno','date']) # sort
ss_permno.head()

ret    shrout      prc
permno  date                                
10145.0 2000-01 -0.167931  789233.0  48.0000
        2000-02  0.006510  795134.0  48.1250
        2000-03  0.094805  796591.0  52.6875
        2000-04  0.062871  796591.0  56.0000
        2000-05 -0.020089  798161.0  54.6875

In [197]:
# change index: first date then permno
ss_date = ss_permno.reset_index()
ss_date = ss_date.set_index(['date','permno']) # multiple index
ss_date = ss_date.sort_values(by=['date','permno']) # sort
ss_date.head()

ret    shrout      prc
date    permno                              
2000-01 10145.0 -0.167931  789233.0  48.0000
        10294.0 -0.072816    4786.0  11.9375
        10308.0 -0.038961    7040.0  27.7500
        10516.0 -0.030928  608360.0  11.7500
        10517.0 -0.078767    3830.0 -16.8125

b)

Using the full sample, estimate the market beta for each stock:

Beta = Cov (R,Rm) / var(Rm)

In [198]:
# list of right permnos (no duplicates):
list_permno = list(stocks_del.loc[~stocks_del.index.duplicated(keep='first')].index)
num_permno = len(list_permno) # 639

In [199]:
market_beta = []
ret_market = rm.mkt_ret.values
var_market = np.var(ret_market) # Var(Rm)
for i in list_permno: # (list_permno): #cycle for each stock (639 elemts)
        current_stocks = stocks_del.loc[i] # long 240 because we have 240 observations for each stock
        ret_cur_stocks = current_stocks.ret.values
        covariance = np.cov(ret_cur_stocks.astype(float), ret_market.astype(float)) # Cov(R,Rm)
        market_beta.append(covariance[0,1]/var_market)

Create a DataFrame where for each stock we have its beta

In [164]:
data = pd.DataFrame(market_beta, columns={'beta'}, index=list_permno) # for each stock we have its beta
#data = data.mul(240) # multiplicate di merge the dataset
data = data.sort_index()
data.head()

,beta
10145.0,1.205212
10294.0,0.761927
10308.0,0.445481
10516.0,0.499793
10517.0,0.645500


I add to the stocks dataframe our beta, creating a new column:

In [165]:
stocks_del = stocks_del.merge(data, left_index=True, right_index=True)
stocks_del['permno'] = stocks_del.index
stocks_with_beta = stocks_del.set_index(['permno','date']) # re-index
stocks_with_beta.head()

ret    shrout      prc      beta
permno  date                                          
10145.0 2000-01 -0.167931  789233.0  48.0000  1.205212
        2000-02  0.006510  795134.0  48.1250  1.205212
        2000-03  0.094805  796591.0  52.6875  1.205212
        2000-04  0.062871  796591.0  56.0000  1.205212
        2000-05 -0.020089  798161.0  54.6875  1.205212

For each month, sort stocks by beta into 10 decile portfolios

In [182]:
decile_beta = beta_group_data.set_index('date').groupby('date')
decile_beta = decile_beta.transform(lambda x: pd.qcut(x, 10))
decile_beta = decile_beta.loc[~decile_beta.index.duplicated(keep='first')]
decile_beta = decile_beta.rename(columns={'beta':'decile'})
intervals = decile_beta.decile.loc[~decile_beta.decile.duplicated(keep='first')] #intervals
decile_beta.head()

,decile
date,
2000-01,"(-0.727, -0.0641]"
2000-02,"(-0.727, -0.0641]"
2000-03,"(-0.727, -0.0641]"
2000-04,"(-0.727, -0.0641]"
2000-05,"(-0.727, -0.0641]"


For each portfolio, compute the equal weighted average return

In [172]:
equal_stocks = stocks_del.set_index('date').drop(columns='beta') # set index date to merge with decile
equal_stocks = equal_stocks.merge(decile_beta, left_index=True, right_index=True) #merge
equal_stocks['date'] = equal_stocks.index # set data as column
equal_stocks = equal_stocks.set_index('permno').sort_values(['decile', 'date'])
equal_stocks.head()

,ret,shrout,prc,decile,date
permno,,,,,
10145.0,-0.167931,789233.0,48.0000,"(-0.727, -0.0641]",2000-01
10294.0,-0.072816,4786.0,11.9375,"(-0.727, -0.0641]",2000-01
10308.0,-0.038961,7040.0,27.7500,"(-0.727, -0.0641]",2000-01
10516.0,-0.030928,608360.0,11.7500,"(-0.727, -0.0641]",2000-01
10517.0,-0.078767,3830.0,-16.8125,"(-0.727, -0.0641]",2000-01
